<a href="https://colab.research.google.com/github/mattiapocci/PhilosoperRank/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Imports and Drive Mount***

In [2]:

#!pip3 install -U spacy[cuda92]
#!python3 -m spacy download en_core_web_lg
#https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

import sys
import time
import os
import numpy as np
import random
import json
import spacy
from spacy import displacy
import nltk
import pprint
from nltk.tokenize import word_tokenize

nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")
spacy.prefer_gpu()

from google.colab import drive

drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# ***Language Analysis***

In [22]:

"""
1. X cite(s|d) Y1, Y2, …, Yn as (an) influence(s)
2. X was influenced by Y1, Y2, …, Yn
3. Y has been cited as an influence by X1, X2, …, Xn
4. Y influence on X1, X2, …, Xn …
5. Y1, Y2, …, Yn influenced hi(m|s) …
"""

doc1 = nlp("Mark was influenced by Jhon")
doc2 = nlp("Jhon influenced Mark")
doc3 = nlp("Mark was an influence of Jhon and Albert")
doc4 = nlp("Jhon influenced him")
doc5 = nlp("Adler was also influenced by the philosophies of Immanuel Kant, Friedrich Nietzsche, Rudolf Virchow and the statesman Jan Smuts ")
doc6 = nlp("He is cited as an influence on John Milton")
print("------")
print("1 Mark was influenced by Jhon")
displacy.render(doc1, style="dep", jupyter=True)
print("CHUNKS")
for chunk in doc1.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc1:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("2 Jhon influenced Mark")
displacy.render(doc2, style="dep", jupyter=True)
print("CHUNKS")

for chunk in doc2.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc2:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("3 Mark was an influence of Jhon and Albert")
displacy.render(doc3, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc3.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc3:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("------")
print("4 Jhon influenced him")
displacy.render(doc4, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc4.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc4:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

print("5 Jhon influenced him")
displacy.render(doc5, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc5.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc5:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
    
print("6 Jhon influenced him")
displacy.render(doc5, style="dep", jupyter=True)
print("CHUNKS")
    
for chunk in doc5.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_, 
            chunk.root.head.text)
print("TREE")
for token in doc5:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

    
#print(jsonlist[0]["article"])
#doc = nlp(jsonlist[0]["article"])

------
1 Mark was influenced by Jhon


CHUNKS
Mark Mark nsubjpass influenced
Jhon Jhon pobj by
TREE
Mark nsubjpass influenced VERB []
was auxpass influenced VERB []
influenced ROOT influenced VERB [Mark, was, by]
by agent influenced VERB [Jhon]
Jhon pobj by ADP []
------
2 Jhon influenced Mark


CHUNKS
Jhon Jhon nsubj influenced
Mark Mark dobj influenced
TREE
Jhon nsubj influenced VERB []
influenced ROOT influenced VERB [Jhon, Mark]
Mark dobj influenced VERB []
------
3 Mark was an influence of Jhon and Albert


CHUNKS
Mark Mark nsubj was
an influence influence attr was
Jhon Jhon pobj of
Albert Albert conj Jhon
TREE
Mark nsubj was AUX []
was ROOT was AUX [Mark, influence]
an det influence NOUN []
influence attr was AUX [an, of]
of prep influence NOUN [Jhon]
Jhon pobj of ADP [and, Albert]
and cc Jhon PROPN []
Albert conj Jhon PROPN []
------
4 Jhon influenced him


CHUNKS
Jhon Jhon nsubj influenced
him him dobj influenced
TREE
Jhon nsubj influenced VERB []
influenced ROOT influenced VERB [Jhon, him]
him dobj influenced VERB []
5 Jhon influenced him


CHUNKS
Adler Adler nsubjpass influenced
the philosophies philosophies pobj by
Immanuel Kant Kant pobj of
Friedrich Nietzsche Nietzsche conj Kant
Rudolf Virchow Virchow conj Nietzsche
the statesman statesman conj Virchow
Jan Smuts Smuts appos statesman
TREE
Adler nsubjpass influenced VERB []
was auxpass influenced VERB []
also advmod influenced VERB []
influenced ROOT influenced VERB [Adler, was, also, by]
by agent influenced VERB [philosophies]
the det philosophies NOUN []
philosophies pobj by ADP [the, of]
of prep philosophies NOUN [Kant]
Immanuel compound Kant PROPN []
Kant pobj of ADP [Immanuel, ,, Nietzsche]
, punct Kant PROPN []
Friedrich compound Nietzsche PROPN []
Nietzsche conj Kant PROPN [Friedrich, ,, Virchow]
, punct Nietzsche PROPN []
Rudolf compound Virchow PROPN []
Virchow conj Nietzsche PROPN [Rudolf, and, statesman]
and cc Virchow PROPN []
the det statesman NOUN []
statesman conj Virchow PROPN [the, Smuts]
Jan compound Smuts PROPN []
Smuts appos statesman NOUN [Jan]
6 J

CHUNKS
Adler Adler nsubjpass influenced
the philosophies philosophies pobj by
Immanuel Kant Kant pobj of
Friedrich Nietzsche Nietzsche conj Kant
Rudolf Virchow Virchow conj Nietzsche
the statesman statesman conj Virchow
Jan Smuts Smuts appos statesman
TREE
Adler nsubjpass influenced VERB []
was auxpass influenced VERB []
also advmod influenced VERB []
influenced ROOT influenced VERB [Adler, was, also, by]
by agent influenced VERB [philosophies]
the det philosophies NOUN []
philosophies pobj by ADP [the, of]
of prep philosophies NOUN [Kant]
Immanuel compound Kant PROPN []
Kant pobj of ADP [Immanuel, ,, Nietzsche]
, punct Kant PROPN []
Friedrich compound Nietzsche PROPN []
Nietzsche conj Kant PROPN [Friedrich, ,, Virchow]
, punct Nietzsche PROPN []
Rudolf compound Virchow PROPN []
Virchow conj Nietzsche PROPN [Rudolf, and, statesman]
and cc Virchow PROPN []
the det statesman NOUN []
statesman conj Virchow PROPN [the, Smuts]
Jan compound Smuts PROPN []
Smuts appos statesman NOUN [Jan]


#***Import JSON articles***


In [0]:
"""
{
    "philosopher": "name",
    "article": "plaintext_article",
    "pageid": "id",
    "table_influenced": 
        [
            {
                "0": "name_of_someone_philosopher_influenced_by"
            }
        ]
    "table_influences":
        [
            {
                "0": "name_of_someone_philosopher_influences"
            }
        ]
}
"""
#open the file
with open('/content/drive/My Drive/folder/result_1.json') as f:
  jsonlist = json.load(f)

# ***Text Preprocessing***

In [92]:
"""
#HIM
#divide in sentences the article

#eliminate sentences that don't include influenc -e, -ed, -es, -ing etc

#pattern matching on patterns (see language analysis)

#Entity recognition and recostruction

#US
"mark influenced jhon. Today is friday"
#divide in sentences the article
"mark influenced jhon"
"Today is friday"
#eliminate sentences that don't include influenc -e, -ed, -es, etc
"mark influenced jhon"

#pattern matching on patterns (using spacy 
?)
[x = "mark", y = "jhon"]

#Entity recognition and recostruction(here we have a proble, when he had Hendrix he could )
"""
import time
start = time.time()

for elem in jsonlist:
    #divide in sentences the article
    sent_list = nltk.sent_tokenize(elem["article"])
    
    sent_list = [word_tokenize(i) for i in sent_list]

    influence_list = []
    influence_declinations = ["influence", "influenced", "influences", "influencing"]

    for word_list in sent_list:
        temp = [x for x in word_list if x in influence_declinations]
        if len(temp) != 0:
            influence_list.append(' '.join(word for word in word_list))

    new_list = []
    for sent in influence_list:
        new_list.append(sent)
        #displacy.render(sent, style="dep", jupyter=True)
    elem["article"] = new_list

print(time.time() - start)

41.20190477371216


# ***Processing Giuliano(Spacy Dependencies)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""


def display_info(list_of_sentences):
    print("------")
    for sent in list_of_sentences:
        print(sent)
        doc = nlp(sent)
        displacy.render(doc, style="dep", jupyter=True)
        print("CHUNKS")

        for chunk in doc.noun_chunks:
            print(chunk.text, chunk.root.text, chunk.root.dep_, 
                    chunk.root.head.text)
        print("TREE")
        for token in doc:
            print(token.text, token.dep_, token.head.text, token.head.pos_,
                    [child for child in token.children])
"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""
"""
import re
print(jsonlist_no_empty[20]["article"][0])
tokens = [i for i in jsonlist_no_empty[20]["article"][0] if i.isupper() ]
print(tokens)
"""

for phil in jsonlist_no_empty:
    display_info(phil["article"])




# ***Processing Mattia(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""

# ***Processing Luigi(Regex)***

In [0]:

"""
Input cella:
jsonlist = 
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            "frase contenente influence",

            "frase contenente influence", etc
        ]
    }
]

Output cella:
output =
[
    {
        "philosopher" = "philosopher_name",
        "pageid" = id,
        "article" = 
        [
            [
                "frase contenente influence",
                [
                    list of influencers
                ],
                [
                    list of influenced
                ]
        ]
    }
]
"""

"""
#some prints and infos

#pprint.pprint(jsonlist)
print(len(jsonlist))
jsonlist_no_empty = [i for i in jsonlist if len(i["article"]) != 0]
print(len(jsonlist_no_empty))
print([[i["philosopher"], len(i["article"]), i["article"]] for i in jsonlist_no_empty if i["philosopher"] == "Socrates"])
"""